In [5]:
import pickle
import numpy as np

In [11]:
import json
import pandas as pd
import os

In [6]:

output = []

with open("./output/09_ocrnet_r101-d8_50e.bin", "rb") as f:
    output_mmseg1 = pickle.load(f)
    
with open("./output/12_hr18_ocr_50e_best50e.bin", "rb") as f:
    output_mmseg2 = pickle.load(f)


for predict1, predict2 in zip(output_mmseg1, output_mmseg2):
    predict = predict1 + predict2
    predict = predict.argmax(axis=0)
    output.append(predict)

output = np.array(output)
print(output.shape)

(624, 512, 512)


In [12]:
# sample_submisson.csv 열기
submission = pd.read_csv('./output/sample_submission.csv', index_col=None)
json_dir = os.path.join("../input/data/test.json")
with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

input_size = 512
output_size = 256
bin_size = input_size // output_size
		
# PredictionString 대입
for image_id, predict in enumerate(output):
    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    
    temp_mask = []
    predict = predict.reshape(1, 512, 512)
    mask = predict.reshape((1, output_size, bin_size, output_size, bin_size)).max(4).max(2) # resize to 256*256
    temp_mask.append(mask)
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)

    string = oms.flatten()

    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(os.path.join('./output/', f'ensemble_temp.csv'), index=False)